## Loading a pre-trained model

Here we load a model that was previously trained and exported. 
<br>There are two files that defines the model:
* __Model Architecture__: Which defines what are the layers that defines this model. Usually a text file, with extension such as ".yaml"
* __Layers Weights__: Contains the actual weights of the layers (W). This is the actual learnt model, and is loaded after the model is created from the architecture file. Weights file extension is ".h5".

In [14]:
import PML 

#load the previously trained model
model=PML.import_model("models/catsdogs")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
__________

Here we load some test data, and check their evaluation:

In [16]:

TARGET_WIDTH=150
TARGET_HEIGHT=150
nb_classes=2

test=PML.load_img_dataset_from_folder("test/catsdogs/",target_size=(TARGET_WIDTH,TARGET_HEIGHT))
labels=['cats','dogs']
norm=PML.Normalizer01(0,255)
PML.test_samples(model,test[0],test[1],labels,norm,False)

0- [dogs] with total of 1012 samples
1- [cats] with total of 1011 samples
Accuracy for [dogs] is 84%
Accuracy for [cats] is 89%


(array([163., 116.]), array([1012., 1011.]))

In [10]:
#setup OSC Protocol to communicate with Processing
import OSCHelper
server=OSCHelper.createServer(9000)
client=OSCHelper.createClient(4200)

Starting server on: 131.113.137.195:9000


In [ ]:
import numpy as np
import io
from PIL import Image

def onOSC_Image(*args):
    print(args[0])
    width=args[1]
    height=args[2]
    jpgdata=args[3]
    file_jpgdata = io.BytesIO(jpgdata)
    img = Image.open(file_jpgdata).resize((TARGET_WIDTH,TARGET_HEIGHT)).convert("L")
    img=np.array(img).astype(float)
    img=img/255.0
    x=PML.prepare_conv_samples([img])[0]
    res=model.predict(np.array([x]))
    label_idx=np.argmax(res)
    client.send_message("/output/label",[int(label_idx),str(labels[label_idx])])
    print(labels[label_idx])

server.addMsgHandler( "/inputs/image", onOSC_Image )

OSCHelper.start_server(server)


In [12]:
server.stop()